In [4]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [19]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
697,So you think a talking parrot is not your cup ...,positive
160,The Salena Incident is set in Arizona where si...,negative
137,"This film is a tapestry, a series of portraits...",positive
181,William Shakespeare's plays are classified as ...,positive
290,This romantic comedy isn't too bad. There are ...,negative


In [20]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vives\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vives\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [22]:
df = normalize_text(df)
df.head()

,review,sentiment
697,think talking parrot cup tea huh well think ag...,positive
160,salena incident set arizona six death row inma...,negative
137,film tapestry series portrait rom community wo...,positive
181,william shakespeare s play classified comedy t...,positive
290,romantic comedy bad funny thing happening ther...,negative


In [23]:
df['sentiment'].value_counts()

sentiment
negative    267
positive    233
Name: count, dtype: int64

In [24]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [25]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
697,think talking parrot cup tea huh well think ag...,1
160,salena incident set arizona six death row inma...,0
137,film tapestry series portrait rom community wo...,1
181,william shakespeare s play classified comedy t...,1
290,romantic comedy bad funny thing happening ther...,0


In [26]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [29]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [31]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Viv-19/MLOPS_Big_Beautifull_Project.mlflow')
dagshub.init(repo_owner='Viv-19', repo_name='MLOPS_Big_Beautifull_Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\vives\miniconda3\envs\ML_DL_venv\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=fa0a777a-758f-431b-9160-4f3cb3b32d81&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=8a071804d00b6b7621beecc67c402120ef65e7dc660363ffc97d227487e5c158




Accessing as Viv-19

Initialized MLflow to track repo "Viv-19/MLOPS_Big_Beautifull_Project"

Repository Viv-19/MLOPS_Big_Beautifull_Project initialized!

2025/07/04 16:20:37 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/eca27207fece4ffd8acccf7f7d135c5e', creation_time=1751626237696, experiment_id='0', last_update_time=1751626237696, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-07-04 17:01:00,893 - INFO - Starting MLflow run...
2025-07-04 17:01:01,423 - WARNING - Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Viv-19/MLOPS_Big_Beautifull_Project.mlflow/api/2.0/mlflow/runs/create
2025-07-04 17:01:02,067 - INFO - Logging preprocessing parameters...
2025-07-04 17:01:03,622 - INFO - Initializing Logistic Regression model...
2025-07-04 17:01:03,625 - INFO - Fitting the model...
2025-07-04 17:01:03,779 - INFO - Model training complete.
2025-07-04 17:01:03,779 - INFO - Logging model parameters...
2025-07-04 17:01:04,157 - INFO - Making predictions...
2025-07-04 17:01:04,159 - INFO - Calculating evaluation metrics...
2025-07-04 17:01:04,197 - INFO - Logging evaluation metrics...
2025-07-04 17:01:09,546 - INFO - Saving and logging the model...
2025/07/04 17:01:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead

🏃 View run painted-fox-856 at: https://dagshub.com/Viv-19/MLOPS_Big_Beautifull_Project.mlflow/#/experiments/0/runs/ca4d520d1d1e457d80de472823c08bb6
🧪 View experiment at: https://dagshub.com/Viv-19/MLOPS_Big_Beautifull_Project.mlflow/#/experiments/0
